In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import joblib

import numpy as np
import pandas as pd
import os

In [10]:
#設定LSTM往前看的筆數和預測筆數
LookBackNum = 12 #LSTM往前看的筆數
ForecastNum = 48 #預測筆數

my_path_example_completeAVG = r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\ExampleTrainData(AVG)"
my_path_IncompleteAVG = r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\ExampleTrainData(IncompleteAVG)"
#載入訓練資料
DataName = os.path.join(my_path_example_completeAVG ,'AvgDATA_17.csv')
SourceData = pd.read_csv(DataName, encoding='utf-8')

#迴歸分析 選擇要留下來的資料欄位
#(風速,大氣壓力,溫度,濕度,光照度)
#(發電量)
Regression_X_train = SourceData[['WindSpeed(m/s)','Pressure(hpa)','Temperature(°C)','Humidity(%)','Sunlight(Lux)']].values
Regression_y_train = SourceData[['Power(mW)']].values

#LSTM 選擇要留下來的資料欄位
#(風速,大氣壓力,溫度,濕度,光照度)
AllOutPut = SourceData[['WindSpeed(m/s)','Pressure(hpa)','Temperature(°C)','Humidity(%)','Sunlight(Lux)']].values

#正規化
LSTM_MinMaxModel = MinMaxScaler().fit(AllOutPut)
AllOutPut_MinMax = LSTM_MinMaxModel.transform(AllOutPut)

X_train = []
y_train = []

#設定每i-12筆資料(X_train)就對應到第i筆資料(y_train)
for i in range(LookBackNum,len(AllOutPut_MinMax)):
  X_train.append(AllOutPut_MinMax[i-LookBackNum:i, :])
  y_train.append(AllOutPut_MinMax[i, :])


X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshaping
#(samples 是訓練樣本數量,timesteps 是每個樣本的時間步長,features 是每個時間步的特徵數量)
X_train = np.reshape(X_train,(X_train.shape [0], X_train.shape [1], 5))

In [ ]:
#============================建置&訓練「LSTM模型」============================
#建置LSTM模型

regressor = Sequential ()


####################

# regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 5)))

# regressor.add(LSTM(units = 64))

# regressor.add(Dropout(0.2))
######################

# 第一層 LSTM
# 調整 units (神經元數量)：可以增加或減少該層的神經元數量以影響模型的表現。
# 嘗試 64、128、256 等不同數值。
regressor.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 5)))

# 第二層 LSTM
regressor.add(LSTM(units=64))  # 嘗試 32、128 等

# Dropout 層：調整 Dropout 比例
regressor.add(Dropout(0.2))  # 嘗試 0.3 或 0.5。

regressor.add(Dense(units=5))

# 編譯模型：調整 optimizer 和 loss
# 嘗試 'adam'、'rmsprop' 或 'sgd'。
regressor.compile(optimizer='adam', loss='mean_squared_error')


c:\Users\andy.lu\AppData\Local\anaconda3\envs\team7012\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
#開始訓練
regressor.fit(X_train, y_train, epochs = 100, batch_size = 128)

#保存模型
from datetime import datetime
NowDateTime = datetime.now().strftime("%Y-%m-%dT%H_%M_%SZ")
LSTMfilename = './model_each_location/LSTM/WheatherLSTM_'+NowDateTime+'.h5'
regressor.save(LSTMfilename)
print('Model Saved')

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0029
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0031
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0029
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0033
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0030
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0033
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0028
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0029
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0032
Epoch 10/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0030
Epoch 11/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0030
Epoch 12/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 7s 168ms/step - loss: 0.0031
Epoch 13/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - loss: 0.0033
Epoch 14/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - loss: 0.0032
Epoch 15/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 100ms/step 

Model Saved


In [14]:
#============================建置&訓練「回歸模型」========================

#開始迴歸分析(對發電量做迴歸)
RegressionModel = LinearRegression()
RegressionModel.fit(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train)

#儲存回歸模型
from datetime import datetime
NowDateTime = datetime.now().strftime("%Y-%m-%dT%H_%M_%SZ")
regression_filename = './model_each_location/LSTM/WheatherRegression_'+NowDateTime
joblib.dump(RegressionModel, regression_filename)

#取得截距
print('截距: ',RegressionModel.intercept_)

#取得係數
print('係數 : ', RegressionModel.coef_)

#取得R平方
print('R squared: ',RegressionModel.score(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train))

截距:  [-221.39953665]
係數 :  [[   0.          135.11381615  117.68809059   23.89564382 1761.56873922]]
R squared:  0.899519098844525


In [ ]:
#============================預測數據============================
#載入模型
regressor = load_model(LSTMfilename)
Regression = joblib.load(regression_filename)


#載入測試資料
DataName = "./36_TestSet_SubmissionTemplate/upload(no answer).csv"
SourceData = pd.read_csv(DataName, encoding='utf-8')
target = ['序號']
EXquestion = SourceData[target].values

inputs = [] #存放參考資料
PredictOutput = [] #存放預測值(天氣參數)
PredictPower = [] #存放預測值(發電量) 

count = 0
while(count < len(EXquestion)):
  print('count : ',count)
  LocationCode = int(EXquestion[count])
  strLocationCode = str(LocationCode)[-2:]
  if LocationCode < 10 :
    strLocationCode = '0'+LocationCode

  DataName = os.path.join(my_path_IncompleteAVG , f"IncompleteAvgDATA_{strLocationCode}.csv")
  SourceData = pd.read_csv(DataName, encoding='utf-8')
  ReferTitle = SourceData[['Serial']].values
  ReferData = SourceData[['WindSpeed(m/s)','Pressure(hpa)','Temperature(°C)','Humidity(%)','Sunlight(Lux)']].values
  
  inputs = []#重置存放參考資料

  #找到相同的一天，把12個資料都加進inputs
  for DaysCount in range(len(ReferTitle)):
    if(str(int(ReferTitle[DaysCount]))[:8] == str(int(EXquestion[count]))[:8]):
      TempData = ReferData[DaysCount].reshape(1,-1)
      TempData = LSTM_MinMaxModel.transform(TempData)
      inputs.append(TempData)

  #用迴圈不斷使新的預測值塞入參考資料，並預測下一筆資料
  for i in range(ForecastNum) :

    #print(i)
    
    #將新的預測值加入參考資料(用自己的預測值往前看)
    if i > 0 :
      inputs.append(PredictOutput[i-1].reshape(1,5))

    #切出新的參考資料12筆(往前看12筆)
    X_test = []
    X_test.append(inputs[0+i:LookBackNum+i])
    
    #Reshaping
    NewTest = np.array(X_test)
    NewTest = np.reshape(NewTest, (NewTest.shape[0], NewTest.shape[1], 5))
    
    predicted = regressor.predict(NewTest)
    PredictOutput.append(predicted)
    PredictPower.append(np.round(Regression.predict(predicted),2).flatten())
  
  #每次預測都要預測48個，因此加48個會切到下一天
  #0~47,48~95,96~143...
  count += 48

#寫預測結果寫成新的CSV檔案
# 將陣列轉換為 DataFrame
df = pd.DataFrame(PredictPower, columns=['答案'])



count :  0


C:\Users\andy.lu\AppData\Local\Temp\ipykernel_20556\2961497883.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  LocationCode = int(EXquestion[count])


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\ExampleTrainData(IncompleteAVG)\\01.csv'

In [ ]:
# 將 DataFrame 寫入 CSV 檔案
df.to_csv('output.csv', index=False) 
print('Output CSV File Saved')